In [ ]:
mkdir midi_songs

In [ ]:
cd midi_songs

/content/midi_songs


In [ ]:
!unzip  /content/midi_songs.zip

Archive:  /content/midi_songs.zip
  inflating: midi_songs/0fithos.mid  
  inflating: midi_songs/8.mid        
  inflating: midi_songs/ahead_on_our_way_piano.mid  
  inflating: midi_songs/AT.mid       
  inflating: midi_songs/balamb.mid   
  inflating: midi_songs/bcm.mid      
  inflating: midi_songs/BlueStone_LastDungeon.mid  
  inflating: midi_songs/braska.mid   
  inflating: midi_songs/caitsith.mid  
  inflating: midi_songs/Cids.mid     
  inflating: midi_songs/cosmo.mid    
  inflating: midi_songs/costadsol.mid  
  inflating: midi_songs/dayafter.mid  
  inflating: midi_songs/decisive.mid  
  inflating: midi_songs/dontbeafraid.mid  
  inflating: midi_songs/DOS.mid      
  inflating: midi_songs/electric_de_chocobo.mid  
  inflating: midi_songs/Eternal_Harvest.mid  
  inflating: midi_songs/EyesOnMePiano.mid  
  inflating: midi_songs/ff11_awakening_piano.mid  
  inflating: midi_songs/ff1battp.mid  
  inflating: midi_songs/FF3_Battle_(Piano).mid  
  inflating: midi_songs/FF3_Third_Phase_

In [ ]:
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56439 sha256=26bbee5a240ba031803b43c31a4775c1d2dbf11abed6c3c94011e4e4ad865b50
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [ ]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("/content/midi_songs/midi_songs/*.mid"):  # Adjust the file path for glob.glob
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try:  # File has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # File has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [ ]:

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output, num_classes=n_vocab)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), recurrent_dropout=0.3, return_sequences=True))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3))
    model.add(LSTM(512, return_sequences=False, recurrent_dropout=0.3))  # Ensure return_sequences=False for the last LSTM layer
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])  # Add metrics for tracking during training


    return model
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=30, batch_size=128, callbacks=callbacks_list)

if __name__ == '__main__':
    train_network()


Parsing /content/midi_songs/midi_songs/bcm.mid
Parsing /content/midi_songs/midi_songs/FFVII_BATTLE.mid
Parsing /content/midi_songs/midi_songs/Kingdom_Hearts_Traverse_Town.mid
Parsing /content/midi_songs/midi_songs/FF3_Third_Phase_Final_(Piano).mid
Parsing /content/midi_songs/midi_songs/roseofmay-piano.mid
Parsing /content/midi_songs/midi_songs/FF4.mid
Parsing /content/midi_songs/midi_songs/Suteki_Da_Ne_(Piano_Version).mid
Parsing /content/midi_songs/midi_songs/traitor.mid
Parsing /content/midi_songs/midi_songs/FF6epitaph_piano.mid
Parsing /content/midi_songs/midi_songs/FF8_Shuffle_or_boogie_pc.mid
Parsing /content/midi_songs/midi_songs/ff1battp.mid
Parsing /content/midi_songs/midi_songs/Gold_Silver_Rival_Battle.mid
Parsing /content/midi_songs/midi_songs/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing /content/midi_songs/midi_songs/redwings.mid
Parsing /content/midi_songs/midi_songs/relmstheme-piano.mid
Parsing /content/midi_songs/midi_songs/Kingdom_Hearts_Dearly_Beloved.mid
Parsing /cont

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Pok\xe9mon: Elite Four (Piano)'>; getting generic Instrument
  warnings.warn(


Parsing /content/midi_songs/midi_songs/FFIX_Piano.mid
Parsing /content/midi_songs/midi_songs/JENOVA.mid
Parsing /content/midi_songs/midi_songs/Eternal_Harvest.mid
Parsing /content/midi_songs/midi_songs/gerudo.mid
Parsing /content/midi_songs/midi_songs/ff7-mainmidi.mid
Parsing /content/midi_songs/midi_songs/sandy.mid
Parsing /content/midi_songs/midi_songs/ultros.mid
Parsing /content/midi_songs/midi_songs/ahead_on_our_way_piano.mid
Parsing /content/midi_songs/midi_songs/braska.mid
Parsing /content/midi_songs/midi_songs/fortresscondor.mid
Parsing /content/midi_songs/midi_songs/Rydia_pc.mid
Parsing /content/midi_songs/midi_songs/decisive.mid
Parsing /content/midi_songs/midi_songs/ff4-airship.mid
Parsing /content/midi_songs/midi_songs/cosmo.mid
Parsing /content/midi_songs/midi_songs/great_war.mid
Parsing /content/midi_songs/midi_songs/BlueStone_LastDungeon.mid
Parsing /content/midi_songs/midi_songs/Rachel_Piano_tempofix.mid
Parsing /content/midi_songs/midi_songs/Life_Stream.mid
Parsing /con

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Martin Rosok Copyright \xa9 1998'>; getting generic Instrument
  warnings.warn(


Parsing /content/midi_songs/midi_songs/ff7themep.mid
Parsing /content/midi_songs/midi_songs/tpirtsd-piano.mid
Parsing /content/midi_songs/midi_songs/ff4pclov.mid
Parsing /content/midi_songs/midi_songs/AT.mid
Parsing /content/midi_songs/midi_songs/DOS.mid
Parsing /content/midi_songs/midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Parsing /content/midi_songs/midi_songs/ff4_piano_collections-main_theme.mid
Parsing /content/midi_songs/midi_songs/Still_Alive-1.mid
Parsing /content/midi_songs/midi_songs/ff4-town.mid
Parsing /content/midi_songs/midi_songs/costadsol.mid
Parsing /content/midi_songs/midi_songs/dayafter.mid
Parsing /content/midi_songs/midi_songs/electric_de_chocobo.mid
Parsing /content/midi_songs/midi_songs/z_aeristhemepiano.mid
Parsing /content/midi_songs/midi_songs/lurk_in_dark.mid
Parsing /content/midi_songs/midi_songs/path_of_repentance.mid
Epoch 1/30
251/359 [===================>..........] - ETA: 24:49 - loss: 5.3321 - accuracy: 0.0227

In [ ]:
/content/midi_songs

In [ ]:
https://bitmidi.com/never-gonna-let-you-go-mid